# 🚀 REGRETTABLE ATTRITION TAHMİN PROJESİ
### End-to-End Machine Learning Pipeline ile Yüksek Performanslı Çalışan Kaybı Tahmini

**Proje Sahibi:** Arzu  
**Amaç:** F1 skoru ile yüksek performanslı çalışanların kaybını tahmin etmek  
**Versiyon:** 2.0 (Kalibrasyon + Dinamik Model Seçimi + Dual-Model Yaklaşımı)

---

In [ ]:
# --- HÜCRE 1: ORTAM KURULUMU VE VERİ YÜKLEME ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

# Görselleştirme ayarları
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Makine Öğrenimi Kütüphaneleri
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import (classification_report, accuracy_score, f1_score,
                             roc_auc_score, precision_recall_curve, confusion_matrix,
                             roc_curve, average_precision_score, brier_score_loss)
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.inspection import permutation_importance
import lightgbm as lgb
from numpy import nan_to_num

print("="*80)
print("🚀 REGRETTABLE ATTRITION TAHMİN PROJESİ")
print("="*80)

# Drive'ı Bağla
drive.mount('/content/drive')

# Veriyi Yükle
file_path = '/content/drive/MyDrive/hr_deneme_2/HR_Analytics.csv'

try:
    df = pd.read_csv(file_path)
    print("\n✅ Veri Başarıyla Yüklendi!")
    print(f"📊 Veri Boyutu: {df.shape}")
    print(f"📋 Toplam Sütun: {len(df.columns)}")
    print(f"👥 Toplam Çalışan: {len(df)}")
except FileNotFoundError:
    print("❌ HATA: Dosya bulunamadı!")
    raise SystemExit

print("\n" + "="*80)
print("İLK 3 KAYIT")
print("="*80)
print(df.head(3))

In [ ]:
# --- HÜCRE 2: KEŞİFSEL VERİ ANALİZİ (EDA) ---
print("\n" + "="*80)
print(">>> KEŞİFSEL VERİ ANALİZİ (EDA)")
print("="*80)

# Eksik değer kontrolü
missing = df.isnull().sum()
if missing.sum() > 0:
    print("\n⚠️ Eksik Değerler:")
    print(missing[missing > 0])
else:
    print("\n✅ Hiç eksik değer yok")

# Attrition analizi
print("\n--- Attrition Dağılımı ---")
attrition_counts = df['Attrition'].value_counts()
print(attrition_counts)
print(f"Ayrılma Oranı: %{(attrition_counts.get('Yes', 0) / len(df) * 100):.2f}")

# Performans dağılımı
print("\n--- Performans Dağılımı ---")
perf_dist = df['PerformanceRating'].value_counts().sort_index()
print(perf_dist)

# Veri türleri
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print(f"\n📊 Kategorik: {len(categorical_cols)}, Sayısal: {len(numerical_cols)}")

# Görselleştirme
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Attrition
attrition_counts.plot(kind='bar', ax=axes[0,0], color=['#2ecc71', '#e74c3c'])
axes[0,0].set_title('Attrition Dağılımı', fontsize=13, fontweight='bold')
axes[0,0].set_ylabel('Çalışan Sayısı')
axes[0,0].tick_params(rotation=0)

# 2. Performance
perf_dist.plot(kind='bar', ax=axes[0,1], color='#3498db')
axes[0,1].set_title('Performans Seviyesi Dağılımı', fontsize=13, fontweight='bold')
axes[0,1].set_ylabel('Çalışan Sayısı')
axes[0,1].tick_params(rotation=0)

# 3. Yaş dağılımı
df['Age'].hist(bins=30, ax=axes[1,0], color='#9b59b6', edgecolor='black')
axes[1,0].set_title('Yaş Dağılımı', fontsize=13, fontweight='bold')
axes[1,0].set_xlabel('Yaş')
axes[1,0].set_ylabel('Frekans')

# 4. Gelir dağılımı
df['MonthlyIncome'].hist(bins=30, ax=axes[1,1], color='#e67e22', edgecolor='black')
axes[1,1].set_title('Aylık Gelir Dağılımı', fontsize=13, fontweight='bold')
axes[1,1].set_xlabel('Aylık Gelir')
axes[1,1].set_ylabel('Frekans')

plt.tight_layout()
plt.show()